In [1]:
from dgp import DGP_Finite
from inference import Inference2, Inference
import numpy as np
import pandas as pd 

# generate multuple sample size, multiple model designs, multiple inference method,
# a single estimator
dgp = DGP_Finite('1',1000)
Yobs, D, A = dgp.get_data()
inf = Inference2(Yobs, D, A, dgp.cluster, dgp.tuple_idx, dgp.tau10)
print(inf.inference('mp'), inf.inference('robust'), inf.inference('clustered'))

def cover_rate(sample_size=1000, modelY='1', ntrials=2000):
    cover = np.zeros((ntrials, 3))
    cf_length = np.zeros((ntrials, 3))
    for i in range(2000):
        dgp = DGP_Finite(modelY, sample_size)
        Yobs, D, A = dgp.get_data()
        inf = Inference2(Yobs, D, A, dgp.cluster, dgp.tuple_idx, dgp.tau10)
        cover[i,0] = inf.inference('mp')
        cf_length[i,0] = inf.se_tau10*19.6*2
        cover[i,1] = inf.inference('robust')
        cf_length[i,1] = inf.se_tau10*19.6*2
        cover[i,2] = inf.inference('clustered')
        cf_length[i,2] = inf.se_tau10*19.6*2
    #tmp = cover[:,0] - cover[:,2]
    #print(tmp[tmp>0], cover[:,0][tmp>0], cover[:,2][tmp>0])
    #print(np.mean(tmp>0))
    return np.mean(cover, axis=0), np.mean(cf_length, axis=0)

1 1 1


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/apply.py:290: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  results[i] = self.f(v)


In [2]:
modelYs = ['1','2','3','4','5','6']
sample_sizes = [50, 100, 200, 500, 1000]
#modelYs = ['1']
#sample_sizes = [1000]

In [3]:
output = np.zeros((len(modelYs)*3,len(sample_sizes)))
cf_output = np.zeros((len(modelYs)*3,len(sample_sizes)))
for i, modelY in enumerate(modelYs):
    for j, n in enumerate(sample_sizes):
        cover, cf = cover_rate(n, modelY)
        output[i*3:i*3+3,j] = cover
        cf_output[i*3:i*3+3,j] = cf
print(output, cf)
#pd.DataFrame(output).to_csv("finite_pop_cover_rate.csv")

[[0.95  ]
 [1.    ]
 [0.9505]]


In [3]:
from joblib import Parallel, delayed
import multiprocessing

sample_sizes = [40, 80, 160, 480, 1000]

qk_pairs = [(q,k) for q in modelYs for k in sample_sizes]
def processInput(qk):
    q, k = qk
    cover, cf = cover_rate(k, q)
    return (q,k,cover,cf)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
output = np.zeros((len(modelYs)*3,len(sample_sizes)))
cf_output = np.zeros((len(modelYs)*3,len(sample_sizes)))
for (q,k,cover,cf) in results:
    i = int(q)-1
    j = sample_sizes.index(k)
    output[i*3:i*3+3,j] = cover
    cf_output[i*3:i*3+3,j] = cf
print(cf_output)
pd.DataFrame(cf_output).to_csv("sup_pop_cf.csv")

[[ 2.09059733  1.29988532  0.88448906  0.50540412  0.35053067]
 [ 5.23699319  3.7559094   2.65925572  1.54373362  1.07086228]
 [ 1.88945522  1.27257969  0.88572448  0.50702789  0.35088164]
 [ 5.03755465  3.43847842  2.40605648  1.38579312  0.95999242]
 [ 7.43937286  5.3316549   3.77710926  2.18493765  1.5149678 ]
 [ 9.78851794  6.70140252  4.64574453  2.64675303  1.82744625]
 [ 4.63943578  3.19280445  2.24160142  1.29275434  0.89540124]
 [ 6.78102924  4.85952735  3.45384519  1.99975768  1.38621195]
 [ 9.02353294  6.17612772  4.29313105  2.44892327  1.69085264]
 [ 4.66767834  3.18989249  2.24252315  1.29276614  0.8962576 ]
 [ 6.33686301  4.52819296  3.22283598  1.86563087  1.29437254]
 [ 9.04884341  6.16725645  4.2942263   2.44887235  1.69246087]
 [19.34535669 10.86693211  6.07844366  2.56419598  1.54149014]
 [25.20717721 18.52031235 13.32864435  7.7258967   5.35501551]
 [18.38784834 11.36629217  7.25818919  3.79970228  2.55173925]
 [ 5.19003935  3.57732239  2.50334612  1.44709115  1.00

In [12]:
output = np.zeros((len(modelYs)*3*2,len(sample_sizes)))
for (q,k,cover,cf) in results:
    i = int(q)-1
    j = sample_sizes.index(k)
    out = [cover[0], cf[0], cover[1], cf[1], cover[2], cf[2]]
    output[i*6:i*6+6,j] = out
print(output)
pd.DataFrame(output).to_csv("finite_pop_cover_rate_cf.csv")

[[ 0.989       0.962       0.9515      0.9425      0.95      ]
 [ 2.29209719  1.30825506  0.94138252  0.51455701  0.34270539]
 [ 1.          1.          1.          1.          1.        ]
 [ 4.82607907  3.49792121  2.60715827  1.54552322  1.05692822]
 [ 0.93        0.94        0.944       0.9495      0.9525    ]
 [ 1.86281347  1.19124958  0.9232686   0.51868016  0.34604489]
 [ 1.          1.          1.          1.          1.        ]
 [ 4.91962864  3.30422349  2.2311134   1.41050385  0.94366332]
 [ 1.          1.          1.          1.          1.        ]
 [ 7.3403132   5.20383954  3.45575539  2.25635425  1.4978796 ]
 [ 1.          1.          1.          1.          1.        ]
 [ 9.51791927  6.33076023  4.2581047   2.7201505   1.79171009]
 [ 1.          1.          1.          1.          1.        ]
 [ 5.0796704   3.24146721  2.30796154  1.31179503  0.89886524]
 [ 1.          1.          1.          1.          1.        ]
 [ 7.27180756  4.95263567  3.59894991  2.03147129  1.38

In [14]:
print(cf_output)
pd.DataFrame(cf_output).to_csv("finite_pop_cf.csv")

[[ 2.29209719  1.30825506  0.94138252  0.51455701  0.34270539]
 [ 4.82607907  3.49792121  2.60715827  1.54552322  1.05692822]
 [ 1.86281347  1.19124958  0.9232686   0.51868016  0.34604489]
 [ 4.91962864  3.30422349  2.2311134   1.41050385  0.94366332]
 [ 7.3403132   5.20383954  3.45575539  2.25635425  1.4978796 ]
 [ 9.51791927  6.33076023  4.2581047   2.7201505   1.79171009]
 [ 5.0796704   3.24146721  2.30796154  1.31179503  0.89886524]
 [ 7.27180756  4.95263567  3.59894991  2.03147129  1.38664455]
 [ 9.80447654  6.28785613  4.44877442  2.49526974  1.69479806]
 [ 4.61258478  3.36821907  2.3006996   1.27558181  0.90030112]
 [ 6.24795853  4.85631838  3.27817306  1.83530835  1.29712359]
 [ 9.13413471  6.49718285  4.39821729  2.4093726   1.69546483]
 [15.80150892  7.86943779  5.74796145  3.60657493  1.57690562]
 [22.88568834 15.94401286 11.82184574  8.84439883  5.4012857 ]
 [16.40033374  9.47734871  7.24880643  4.40792115  2.51964466]
 [ 5.4851569   3.62880492  2.59623206  1.43854915  1.02